# Overview
This notebook explores boundary behavior of the various constraint enforcement algorithms.

In [11]:
include("../../src/MetaDoE.jl")
using .MetaDoE: Experiments, ConstraintEnforcement, Constraints, PSO, Objectives, Designs
using Base.Iterators
using BenchmarkTools
using Polyhedra
using HiGHS

In [3]:
function run_experiments(objectives, enforcers, save_func)
    for (obj, enf) in Iterators.product(objectives, enforcers)
        println("Objective: $obj, Enforcer: $enf")
        context = PSO.create_context(
            experiment, 
            obj; 
            callback = PSO.aggregate_results(;save_world=true), 
            runner_params=runner_params,
            enforcer_type = enf
        )
        runner_state, history = PSO.optimize(context)
        save_func(history, obj, enf)
        println("Best score: $(runner_state.swarm.memory.global_best_score)")
    end
end

function save_3d_results(; prefix = "")
    return (history, obj, enf) -> PSO.save_history_3d(history; location = "$(prefix)_$(obj)_$(enf).npy")
end

save_3d_results (generic function with 1 method)

In [4]:
N = 200
K = 50
bound = 100

A, b = Constraints.hypercube(K, bound)
og_experiment = Experiments.create(N, K)
og_experiment = Experiments.with_linear_constraints(og_experiment, A, b)
for i in 1:(K-1)
    og_experiment = Experiments.with_factor_ratio(og_experiment, i, K, 2.0)
end

In [51]:
# Boundary proximal
n = 10
δ = n * eps()
t = vcat(ones(K - 1) * δ * sqrt(5), [0])
translated_b = og_experiment.constraints.b .+ og_experiment.constraints.A * t
experiment = Experiments.create(N, K)
experiment = Experiments.with_linear_constraints(experiment, og_experiment.constraints.A, translated_b)

Main.MetaDoE.Experiments.Experiment(Dict("32" => 32, "29" => 29, "1" => 1, "12" => 12, "48" => 48, "20" => 20, "2" => 2, "6" => 6, "41" => 41, "27" => 27…), Main.MetaDoE.ConstraintEnforcement.LinearConstraints([1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 -2.0; 0.0 0.0 … 1.0 -2.0], [100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0, 100.0  …  4.965068306494546e-15, 4.965068306494546e-15, 4.965068306494546e-15, 4.965068306494546e-15, 4.965068306494546e-15, 4.965068306494546e-15, 4.965068306494546e-15, 4.965068306494546e-15, 4.965068306494546e-15, 4.965068306494546e-15]), 200, 50)

In [15]:
# Centroid
p = polyhedron(hrep(og_experiment.constraints.A, og_experiment.constraints.b), DefaultLibrary{Float64}(HiGHS.Optimizer))
center, radius = chebyshevcenter(p)
translated_b = og_experiment.constraints.b .- og_experiment.constraints.A * center
experiment = Experiments.create(N, K)
experiment = Experiments.with_linear_constraints(experiment, og_experiment.constraints.A, translated_b)

Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+02, 1e+02]
Presolving model
149 rows, 51 cols, 347 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-149); columns 0(-51); elements 0(-347) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  5.7294901688e+01
HiGHS run time      

Main.MetaDoE.Experiments.Experiment(Dict("32" => 32, "29" => 29, "1" => 1, "12" => 12, "48" => 48, "20" => 20, "2" => 2, "6" => 6, "41" => 41, "27" => 27…), Main.MetaDoE.ConstraintEnforcement.LinearConstraints([1.0 0.0 … 0.0 0.0; 0.0 1.0 … 0.0 0.0; … ; 0.0 0.0 … 0.0 -2.0; 0.0 0.0 … 1.0 -2.0], [142.70509831248424, 142.70509831248424, 142.70509831248424, 142.70509831248424, 142.70509831248424, 142.70509831248424, 142.70509831248424, 142.70509831248424, 142.70509831248424, 142.70509831248424  …  128.1152949374527, 128.1152949374527, 128.1152949374527, 128.1152949374527, 128.1152949374527, 128.1152949374527, 128.1152949374527, 128.1152949374527, 128.1152949374527, 128.1152949374527]), 200, 50)

In [19]:
runner_params = PSO.runner_params(200, 200, 1e-6)
pso_params = PSO.create_hyperparams(100)

objectives = [Objectives.ackley]
enforcers = [ConstraintEnforcement.Penalty, ConstraintEnforcement.Parametric]
run_experiments(objectives, enforcers, save_3d_results(prefix="../data/10kd_centroid_solution"))

Objective: ackley, Enforcer: Penalty
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 1e+02]
  Cost   [0e+00, 0e+00]
  Bound  [0e+00, 0e+00]
  RHS    [1e+00, 1e+00]
Presolving model
Problem status detected on presolve: Infeasible
Model   status      : Infeasible
Objective value     :  0.0000000000e+00
HiGHS run time      :          0.00
ERROR:   No LP invertible representation for getDualRay
Running HiGHS 1.8.0 (git hash: fcfb534146): Copyright (c) 2024 HiGHS under MIT licence terms
Coefficient ranges:
  Matrix [1e+00, 2e+00]
  Cost   [1e+00, 1e+00]
  Bound  [0e+00, 0e+00]
  RHS    [6e+01, 1e+02]
Presolving model
149 rows, 51 cols, 347 nonzeros  0s
0 rows, 1 cols, 0 nonzeros  0s
0 rows, 0 cols, 0 nonzeros  0s
Presolve : Reductions: rows 0(-149); columns 0(-51); elements 0(-347) - Reduced to empty
Solving the original LP from the solution after postsolve
Model   status      : Optimal
Objective value     :  